In [ ]:
# excel 파일에 저장된 대본의 문장을 모아서 토크나이저를 만든다.
# excel 파일을 제대로 읽기 위해서 openpyxl을 설치
# pip install openpyxl

import pandas as pd
from speechbrain.tokenizers.SentencePiece import SentencePiece
import os
import re

In [ ]:
# speechbrain tokenizer prepare에 있는 nomalize 함수

def normalize(string):
    """
    This function normalizes a given string according to
    the normalization rule
    The normalization rule removes "/" indicating filler words,
    removes "+" indicating repeated words,
    removes all punctuation marks,
    removes non-speech symbols,
    and extracts orthographic transcriptions.

    Arguments
    ---------
    string : str
        The string to be normalized

    Returns
    -------
    str
        The string normalized according to the rules

    """
    # extracts orthographic transcription
    string = re.sub(r"\(([^)]*)\)\/\(([^)]*)\)", r"\1", string)
    # removes non-speech symbols
    string = re.sub(r"n/|b/|o/|l/|u/", "", string)
    # removes punctuation marks
    string = re.sub(r"[+*/.?!,]", "", string)
    # removes extra spaces
    string = re.sub(r"\s+", " ", string)
    string = string.strip()

    return string

In [ ]:
# excel file 읽기, 시이트가 여러개 있는 경우 각각의 시이트를 차례로 읽어온다.
# sheet_num 변수에 읽어올 이이트의 수를 저장, 0,1,...sheet_num-1 순서로 읽어온다.

data_folder = '../raw_data'
output_folder = 'results'
file_name = '따라말하기_강원도_1~3세트_0924.xlsx'

data_file_path = os.path.join(data_folder, file_name)

sheet_num = 3

for i in range(sheet_num):
    # globals()['df_{}'.format(i)] = pd.read_excel(file_name, sheet_name=i, engine='openpyxl') # 아래와 동일
    globals()[f'df_{i}'] = pd.read_excel(data_file_path, sheet_name=i, engine='openpyxl') # df_0, df_1, df_2 에 각각의 내용을 저장

In [ ]:
total_df = pd.concat([
    df_0[['주제', '방언 표기', '표준어 표기']],
    df_1[['주제', '방언 표기', '표준어 표기']],
    df_2[['주제', '방언 표기', '표준어 표기']]])

In [ ]:
total_df['wrd'] = total_df['방언 표기'].apply(normalize)

In [ ]:
total_df

In [ ]:
# total_df.columns = ['subject', 'wrd', 'standard']

In [ ]:
csv_file_name = 'gw.csv'
output_path = os.path.join(output_folder, csv_file_name)

total_df.to_csv(output_path)

In [ ]:
model_dir = '../models/token'

sp = SentencePiece(
    model_dir=model_dir,
    vocab_size=1500,
    annotation_train=output_path,
    annotation_read='wrd',
    model_type='unigram',
    character_coverage=1.0,
    bos_id=1,
    eos_id=2
)